In [1]:
# SU + ANN + CA
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
dataset_1 = pd.read_csv("modiasd.csv")

In [3]:
Y = dataset_1["class"]

In [4]:
X = dataset_1.drop(["class"], axis=1)

In [5]:
#https://github.com/jundongl/scikit-feature
from skfeature.utility.mutual_information import su_calculation
su_scores = [];
for column in X.columns:
    su_scores.append({"column" :column, "value": su_calculation(X[column].as_matrix(),Y.as_matrix())})

c:\users\1585575\desktop\1585575\murali\ml\autism_prediction\autism_venv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [6]:
su_scores.sort(key=lambda x: x["value"], reverse=True)

In [7]:
selected_features = [x["column"] for x in su_scores if x["value"] > 0]

In [8]:
selected_features

['stereotyped behaviors',
 'Severe intellectual disability (IQ < 35)',
 'interact with other people',
 'notmoveindepe',
 'repeating words',
 'repeating actions over and over',
 'What activity or event preceded the behavior?',
 'Vision (Smell) - Sense - Under-Sensitive',
 'Vision (Movement) - Sense - Under-Sensitive',
 'Change routine',
 'Increase sensorimotor skills',
 'Warning signs or predictors of the behavior',
 'Eye aversion',
 'Pause',
 'Vision (sight) - Sense - Under-Sensitive',
 'Vision (Touch (Tactile)) - Sense - Under-Sensitive',
 'Low Tech/No Tech Adaptations for the Visual Environment-Decrease clutter',
 'Lack of empathy(L-1:M-2:H-3)',
 'Total score ? 30',
 'Childhood Autism Rating Scale-Tokyo Version (L=1;M=2;H=3)',
 'obsessively following routines',
 'schedules for their actions',
 'Louder voice',
 'understand gestures such as waving',
 'Routines or repetitive behaviors',
 'want to have routines where things stay the same so they know what to expect.',
 'Vision (sight) - 

In [9]:
X = X[selected_features]
len(X.columns)
num_features = len(selected_features)

In [81]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=100)

In [87]:
from keras.optimizers import Optimizer
from keras import backend as K
import numpy as np
if K.backend() == 'tensorflow':
    import tensorflow as tf
class COCOB(Optimizer):
    """Coin Betting Optimizer from the paper:
        https://arxiv.org/pdf/1705.07795.pdf
    """
    def __init__(self, alpha=100, **kwargs):
        """
        Initialize COCOB Optimizer
        Args:
            alpha: Refer to paper.
        """
        super(COCOB, self).__init__(**kwargs)
        self._alpha = alpha
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
    def get_updates(self, params, loss, contraints=None):
        #print(len(params[0].numpy()))
        self.updates = [K.update_add(self.iterations, 1)]
        grads = self.get_gradients(loss, params)
        shapes = [K.int_shape(p) for p in params]
        L = [K.variable(np.full(fill_value=1e-8, shape=shape)) for shape in shapes]
        reward = [K.zeros(shape) for shape in shapes]
        tilde_w = [K.zeros(shape) for shape in shapes]
        gradients_sum = [K.zeros(shape) for shape in shapes]
        gradients_norm_sum = [K.zeros(shape) for shape in shapes]
        for p, g, li, ri, twi, gsi, gns in zip(params, grads, L, reward,
                                             tilde_w,gradients_sum,
                                               gradients_norm_sum):
            grad_sum_update = gsi + g
            grad_norm_sum_update = gns + K.abs(g)
            l_update = K.maximum(li, K.abs(g))
            reward_update = K.maximum(ri - g * twi, 0)
            new_w = - grad_sum_update / (l_update * (K.maximum(grad_norm_sum_update + l_update, self._alpha * l_update))) * (reward_update + l_update)
            param_update = p - twi + new_w
            tilde_w_update = new_w
            self.updates.append(K.update(gsi, grad_sum_update))
            self.updates.append(K.update(gns, grad_norm_sum_update))
            self.updates.append(K.update(li, l_update))
            self.updates.append(K.update(ri, reward_update))
            self.updates.append(K.update(p, param_update))
            self.updates.append(K.update(twi, tilde_w_update))
            print(self.updates)
        return self.updates
    def get_config(self):
        config = {'alpha': float(K.get_value(self._alpha)) }
        base_config = super(COCOB, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [88]:
model = Sequential()
model.add(Dense(num_features, input_dim=num_features, activation='relu'))
model.add(Dense(num_features, activation='relu'))
model.add(Dense(num_features, activation='relu'))

In [89]:
model.add(Dense(num_features, activation='relu'))
model.add(Dense(num_features, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [90]:
model.compile(loss='binary_crossentropy', optimizer=COCOB(), metrics=['accuracy'])

In [91]:
model.fit(X_train, Y_train, epochs=5, batch_size=10)

[<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       ...,
       [1.e-08, 1.e-08

       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]
[<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e

       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]
[<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'UnreadVariable' shape=(49, 49) dtype=float32, numpy=
array([[1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e-08, 1.e-08, 1.e-08],
       [1.e-08, 1.e-08, 1.e-08, ..., 1.e

Epoch 1/5
191/191 [==============================] - 0s 974us/step - loss: 1.6483 - accuracy: 0.3770
Epoch 2/5
191/191 [==============================] - 0s 183us/step - loss: 1.5980 - accuracy: 0.0838
Epoch 3/5
191/191 [==============================] - 0s 199us/step - loss: 1.6699 - accuracy: 0.0890
Epoch 4/5
191/191 [==============================] - 0s 194us/step - loss: 1.6250 - accuracy: 0.0838
Epoch 5/5
191/191 [==============================] - 0s 183us/step - loss: 1.6283 - accuracy: 0.0838


In [16]:
# evaluate the keras model
_, accuracy = model.evaluate(X_train, Y_train)
print('Accuracy: %.2f' % (accuracy*100))

191/191 [==============================] - 0s 126us/step
Accuracy: 91.62


In [17]:
predictions = model.predict_classes(X_test)

In [18]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

83/83 [==============================] - ETA:  - 0s 60us/step
Accuracy: 90.36
